# 텍스트의 분포를 이용해 벡터화하기

### 목표

1. 단어의 빈도를 이용해 텍스트를 표현하는 방법 알기
2. 텍스트의 분포를 이용해 텍스트를 토큰화하는 방법 알기

### 목차

1. [단어 빈도를 이용한 벡터화 1) Bag of Words](#단어-빈도를-이용한-벡터화-1\)-Bag-of-Words)
2. [단어 빈도를 이용한 벡터화 2) DTM과 코사인 유사도](#단어-빈도를-이용한-벡터화-2\)-DTM과-코사인-유사도)
3. [단어 비도를 이용한 벡터화 3) TF-IDF](#단어-빈도를-이용한-벡터화-3\)-TF-IDF)
4. [4) LSA(Latent Semantic Analysis)](#4\)-LSA(Latent-Semantic-Analysis))
5. [5) LDA(Latent Dirichlet Allocation)](#5\)-LDA(Latent-Dirichlet-Allocation))
6. [텍스트 분포를 이용한 비지도 학습 토크나이저](텍스트-분포를-이용한-비지도-학습-토크나이저)


<hr>

In [38]:
%config Completer.use_jedi = False

## 단어 빈도를 이용한 벡터화 1) Bag of Words

### - 벡터화 방법
1. 통계와 머신러닝을 활용한 방법 <- 오늘 공부할 것!
2. 인공 신경망을 확용하는 방법


### - Bag of Words

- 문서를 단어들의 가방으로 가정
- 문서에 등장하는 텍스트를 단어 단위로 토큰화한 후, 순서를 섞어 중복된 단어들을 제거하지 않고 그 빈도를 카운트한다.
- 어순에 따라 달라지는 의미를 반영하지 못한다는 한계 존재

![image](https://user-images.githubusercontent.com/80008411/135186798-5284ceef-a85f-4565-b928-e8066a398ebd.png)

### - 구현

1. keras.preprocessing.text.Tokenizer 이용
    - Tokenizer.fit_on_texts()
    - tokenizer.word_counts()
    
2. sklearn.feature_extraction.text.CountVectorizer 이용
    - CountVectorizer()
    - CountVectorizer.fit_transform()

In [1]:
# 1. keras 이용
from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()

# BoW 생성
tokenizer.fit_on_texts(sentence)

# bow 변수에 각 단어와 그 빈도를 저장
bow = dict(tokenizer.word_counts)

print("Bag of Words:", bow)
# unique 단어 개수 -> 단어장: 중복 제거한 단어들의 집합
print('단어장 크기:', len(tokenizer.word_counts))

Bag of Words: {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장 크기: 10


In [3]:
# 2. sklearn 이용
from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()

# 각 단어의 빈도수를 기록
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words:', bow)
print('각 단어의 인덱스:', vector.vocabulary_)

# 단어장 크기
print('단어장 크기:', len(vector.vocabulary_))

Bag of Words: [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스: {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}
단어장 크기: 10


- also의 인덱스가 0이니까 빈도가 1
- likes의 인덱스가 4, 빈도는 3

<hr>

## 단어 빈도를 이용한 벡터화 2) DTM과 코사인 유사도

### - DTM(Document-Term Matrix)

- 직역하면 문서-단어 행렬
- 여러 문서의 BoW를 하나의 행렬로 구현한 것
- 즉 각 문서에 등장한 단어의 빈도수를 하나의 행렬로 통합한다.
- rows: 문서, columns: 단어
- 반대의 경우에는 TDM이라고 바꿔서 부른다.

![image](https://user-images.githubusercontent.com/80008411/135188664-c8c543aa-fce5-46de-a2d4-98d232a7a6c5.png)

### - 문서 간 유사도 구하기

- DTM으로 무얼 할 수 있나? = 문서들 간의 유사도 구하기!
- 코사인 유사도 이용

### - 코사인 유사도(Cosine Similarity)

- 두 벡터 간의 코사인 각도를 이용해 구할 수 있는 두 벡터의 유사도
- 두 벡터의 방향이 완전히 동일한 경우 1, 직각을 이루면 0, 정반대 방향이면 -1

![image](https://user-images.githubusercontent.com/80008411/135190408-c8a7024b-1742-46ba-8e12-d6a4bf9ffd81.png)

![image](https://user-images.githubusercontent.com/80008411/135190448-c803fddb-91c1-4141-b4dd-a708bf9baa21.png)


In [4]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

'''
문서1 : 저는 사과 좋아요
문서2 : 저는 바나나 좋아요
문서3 : 저는 바나나 좋아요 저는 바나나 좋아요
'''

doc1 = np.array([0,1,1,1])
doc2 = np.array([1,0,1,1])
doc3 = np.array([2,0,2,2])

def cos_sim(A, B):
    return dot(A,B)/(norm(A)*norm(B))

![image](https://user-images.githubusercontent.com/80008411/135190564-811d78ed-a426-44ab-b262-d5a094b090f0.png)


In [5]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


- 문서3은 문서2에 비해 단지 모든 단어의 빈도수가 1씩 증가했을 뿐
- 즉 문서 내 모든 단어의 빈도수가 동일하게 증가할 경우 기존 문서와의 코사인 유사도가 1이 된다.
- 즉 문서의 길이가 달라도 단어를 기반으로 공정하게 비교할 수 있다.
- 이는 벡터의 크기가 아니라 벡터의 방향(패턴)에 초점을 두기 때문이며, 벡터 내적을 통해 유사도를 구하는 방법과의 차이점이 된다.

### - DTM의 구현

- sklearn.feature_extraction.text.CountVectorize 이용
- 위 BoW 만드는 방법과 동일하나, 단지 다수의 문서를 입력 값으로 주면 된다.

### - DTM의 한계점

- DTM은 BoW 기반으로 문서를 비교할 수 있는 행렬
- 한계점 1. 차원의 저주
    - 문서, 단어의 수가 늘어날수록 행과 열 대부분이 0의 값을 가지며 저장 공간의 낭비가 커짐
- 한계점 2. 단어의 빈도에만 집중
    - 예를 들어 영어 데이터에서 'the'는 어떤 문서에서나 매우 자주 등장한다.
    - 그런데 단지 'the'가 많다는 이유로 어떤 두 문서가 유사하다고 보는 것은 부적절하다.
    - [관련 동영상](https://youtu.be/Rd3OnBPDRbM)

In [6]:
# DTM 구현
corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games',    
]
vector = CountVectorizer()

# 각 문장의 단어 빈도수 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스를 저장
print(vector.vocabulary_)

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


<hr>

## 단어 비도를 이용한 벡터화 3) TF-IDF

### - TF-IDF(Term Frequency-Inverse Document Frequency)

- 직역하면 단어 빈도-역문서 빈도
- 단어의 중요도를 판단해 가중치를 주는 방법
- 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하고, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단한다.
- 그러나 TF-IDF가 항상 DTM보다 성능이 뛰어난 것은 아니다.
- TF-IDF를 사용하려면 우선 DTM을 만든 후에 TF-IDF 가중치를 DTM에 적용해야 한다.

### - TF-IDF 계산하기

1. TF 계산
- y는 문서, x는 단어, TF는 각 문서에 등장하는 단어의 빈도를 의미
- 즉 TF는 DTM을 만들면 자연스럽게 해결된다.

2. IDF 계산
- TF 뒤에 곱해지는 로그항이 IDF를 의미
- IDF를 구하기 위해서는 우선 문서 빈도 DF와 전체 문서의 수 N을 이해해야 한다.
- 예를 들어, 전체 5개의 문서가 있다. 단어 like가 문서2에서 200번, 문서3에서 300번 등장했다. 이때 like의 DF는 2. like가 어떤 문서에서 몇 번 등장했는지가 아니라 몇 개의 문서에서 등장했는지를 카운트한다.

- 따라서 IDF는 $log5/2 = 0.91629072187$

- 문서2의 like의 TF-IDF 값은 $200 * ln5/2 = 183.258146375$

- 문서3의 like의 TF-IDF 값은 $300*ln5/2 = 274.887219562$

- 즉 특정 문서인 문서3에서만 자주 등장하므로 like의 중요도가 높다고 판단할 수 있다.

![image](https://user-images.githubusercontent.com/80008411/135191946-89184b12-0af7-414d-aa8f-c475d180bb86.png)

### - 구현

1. math.log, pandas 이용
    - log 항과 log항의 분모에 각각 1을 더해서 분모가 0이 되지 않도록 하고, log의 진수가 1이 되어 IDF 값이 0이 되지 않도록 한다.
2. sklearn.feature_extraction.text.TfidfVectorizer 이용

In [1]:
# 1. math.log, pandas 이용
from math import log
import pandas as pd

docs = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

# 통합 단어장 만들기
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장 크기:', len(vocab))
print(vocab)

단어장 크기: 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


In [8]:
N = len(docs)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))+1

def tfidf(t, d):
    return tf(t, d) * idf(t)

In [9]:
result = []

# 각 문서에 대해 아래 명령 수행
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tf(t, d))
        
tf_ = pd.DataFrame(result, columns=vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [10]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns=["IDF"])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [11]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


In [16]:
# sklearn.TFidVectorizer 이용
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

tfidfv = TfidfVectorizer().fit(corpus)

# 단어장을 리스트로 저장
vocab = list(tfidfv.vocabulary_.keys())

# 단어장을 알파벳 순으로 정렬
vocab.sort()

# TF-IDF 행렬에 단어장을 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns=vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


### - TF-IDF의 한계점

1. Only based on Terms(words)
2. Weak on capturing document topic
3. Weak handling synonym(different words but same meaning)
- 단어의 의미를 벡터로 표현하지 못함

### - 극복 방법

1. LSA(Latent Semantic Analysis)
2. Word Embedding(Word2Vec, Glove)
3. ConceptNet(knowlege graph 이용)

<hr>

## 4) LSA(Latent Semantic Analysis)

- 잠재 의미 분석
- 전체 corpus에서 문서 속 단어들 사이 관계를 찾아내는 자연어 처리 정보 검색 기술
- 즉 단어와 단어 사이, 문서와 문서 사이, 단어와 문서 사이의 의미적 유사성 점수를 계산할 수 있다.

### - 행렬의 종류
- LSA를 이해하려면 먼저 SVD를 이해해야 하고, SVD를 이해하려면 먼저 행렬의 종류를 알아야 한다.
- 전치 행렬(Transposed Matrix)
    - 원래의 행렬에서 행과 열을 서로 맞바꾼 행렬
![image](https://user-images.githubusercontent.com/80008411/135362121-4ef7c598-7b2f-43d8-af20-c8af9fe742f5.png)

- 단위 행렬(Identity Matrix)
    - 주대각선의 성분이 모두 1이며, 나머지는 모두 0인 정사각 행렬
![image](https://user-images.githubusercontent.com/80008411/135362176-1c19b201-380e-4400-ab0e-17a3891a96e2.png)

- 역행렬(Inverse Matrix)
    - A와 B 행렬을 곱했을 때 결과가 단위 행렬이면, B 행렬을 A의 역행렬이라고 한다.

- 직교 행렬(Orthogonal Matrix)
    - 행렬 A와 A의 전치 행렬을 곱했을 때 단위 행렬이 되면, 이 A를 직교 행렬이라고 한다.
    
- 정방 행렬(Square Matrix)
    - 열과 행의 개수가 동일한 행렬 = 정사각 행렬

### - 특이값 분해(Singular Value Decompotion)

- 정방행렬은 고유 분해로 고유값과 고유벡터를 찾을 수 있다. 정방행렬이 아닌 행렬은 고유 분해가 불가능하기 때문에 대신 그와 비슷한 특이 분해를 한다.
- m x n 크기의 임의의 사각 행렬 A를 특이 벡터(singular vector)의 행렬과 특이값(singular value)의 대각 행렬로 분해하는 것
- 즉 m x n 크기 행렬 A를 다음과 같은 3개 행렬의 곱으로 나타내는 것이 특이값 분해이다.

$$A = U \sum V ^T$$
- 이때 $U, \sum, V$는 다음 조건을 만족해야 한다.
    - 대각 성분이 양수인 대각행렬이어야 한다.큰 수부터 작은 수 순서로 배열한다.
    - U는 M차원 정방행렬로 모든 열벡터가 단위 벡터이고 서로 직교해야 한다.
    - V는 N차원 정방행렬로 모든 열벡터가 단위 벡터이고 서로 직교해야 한다.
- 위 조건들을 만족하는 행렬 $\sum$의 대각 성분들을 특이값, 행렬 $U$의 열벡터들을 왼쪽 특이 벡터, 행렬 $V$의 행벡터들을 오른쪽 특이벡터라고 부른다.


![image](https://user-images.githubusercontent.com/80008411/135362771-04092c35-c144-4c68-b441-e30d8c170733.png)

#### * Truncated SVD

- 특이값 중 가장 큰 = 가장 중요한 t개만 남기고 해당 특이값에 대응되는 특이 벡터들로 행렬 A를 근사(approximate)하도록 하는 것
- 즉 행렬 $\sum$의 대각 원소들 중에서 상위 t개만 남고, U와 V 행렬의 t열까지만 남는다.
- 따라서 세 행렬에서 정보의 손실이 일어나 기존 행렬 A를 정확히 복구할 수는 없다.
- 여기서 t는 하이퍼파라미터로, 클수록 기존 행렬 A의 정보를 많이 남길 수 있지만, 노이즈를 제거하려면 t를 작게 잡아야 한다.

### - LSA 구현

- 토큰화 -> 역토큰화 -> SVD
    - nltk 토크나이저
    - sklearn CountVectorizer 이용해 역토큰화
    - sklearn.decomposition import TruncatedSVD 이용

In [2]:
import urllib.request

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [27]:
# 토큰화 -> 역토큰화
# nltk 데이터셋 다운로드
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /aiffel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /aiffel/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
# !mv ~/nltk_data ~/aiffel/aiffel_projects/goingdeeper/GD3_topic_modeling/data

In [9]:
import os

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename='abcnews-date-text.csv')

('abcnews-date-text.csv', <http.client.HTTPMessage at 0x7ff146037090>)

In [10]:
!mv ~/aiffel/aiffel_projects/goingdeeper/GD3_topic_modeling/abcnews-date-text.csv ~/aiffel/aiffel_projects/goingdeeper/GD3_topic_modeling/data

In [18]:
csv_file = os.getenv('HOME')+'/aiffel/aiffel_projects/goingdeeper/GD3_topic_modeling/data/abcnews-date-text.csv'

In [19]:
data = pd.read_csv(csv_file, error_bad_lines=False)
data.shape

(1082168, 2)

In [20]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [21]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [22]:
text.nunique()

headline_text    1054983
dtype: int64

In [23]:
text.drop_duplicates(inplace=True)
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

In [28]:
# 데이터 정제 및 정규화
# nltk 토크나이저 이용
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)
text.head()

,headline_text
0,"[aba, decides, against, community, broadcastin..."
1,"[act, fire, witnesses, must, be, aware, of, de..."
2,"[a, g, calls, for, infrastructure, protection,..."
3,"[air, nz, staff, in, aust, strike, for, pay, r..."
4,"[air, nz, strike, to, affect, australian, trav..."


In [29]:
# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop_words)])

text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [30]:
# 의미는 같지만 다른 표현의 단어들을 하나로 통합 = lemmatization
# 3인칭 단수 -> 1인칭, 과거형 -> 현재형 등
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1~2인 단어 제거
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) >2])
text[:5]

0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object

In [33]:
# 역토큰화
detokenized = []
for i in range(len(text)):
    t = ' '.join(text[i])
    detokenized.append(t)
    
train_data = detokenized
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

In [34]:
# 상위 5000개 단어만 사용해서 DTM 생성
c_vectorizer = CountVectorizer(stop_words='english', max_features=5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

In [35]:
print('행렬 크기:', document_term_matrix.shape)

행렬 크기: (1054983, 5000)


In [36]:
# sklearn.decomposition.TruncatedSVD.fit_transform()
# 토픽 수 = k를 10으로 설정
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components=n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20541783e-02, -3.83793498e-03,  1.83066586e-02, ...,
        -3.79585091e-03,  6.33471516e-03,  1.33750544e-02],
       [ 2.90489722e-02, -1.09905036e-02,  1.82877259e-02, ...,
         6.72187458e-05, -6.12460820e-03, -7.18113972e-03],
       [ 5.02744537e-03, -2.03619311e-03,  9.85711409e-03, ...,
         2.15992280e-03,  1.72062969e-03,  2.32930166e-03],
       ...,
       [ 2.97127465e-02,  4.90318524e-03,  2.48411762e-02, ...,
        -2.94935638e-02,  1.30609068e-02,  1.98435113e-02],
       [ 6.15917901e-02, -4.05998464e-03,  1.34937629e-01, ...,
        -6.83021626e-01,  9.38331263e-01, -3.41615749e-01],
       [ 7.15454295e-02,  2.62962181e-02,  4.37033563e-03, ...,
         2.99586351e-02,  1.07878139e-02,  2.56091992e-03]])

In [37]:
print(lsa_model.components_.shape)

(10, 5000)


In [39]:
terms = c_vectorizer.get_feature_names() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74635), ('man', 0.45354), ('charge', 0.21092), ('new', 0.14092), ('court', 0.1115)]
Topic 2: [('man', 0.69393), ('charge', 0.30066), ('court', 0.16767), ('face', 0.11333), ('murder', 0.10759)]
Topic 3: [('new', 0.83688), ('plan', 0.23723), ('say', 0.18333), ('govt', 0.11077), ('council', 0.11037)]
Topic 4: [('say', 0.73925), ('plan', 0.35727), ('govt', 0.16524), ('council', 0.12746), ('urge', 0.07676)]
Topic 5: [('plan', 0.73277), ('council', 0.17472), ('govt', 0.13423), ('urge', 0.09044), ('fund', 0.06118)]
Topic 6: [('govt', 0.53639), ('court', 0.27594), ('urge', 0.26491), ('fund', 0.20883), ('face', 0.16658)]
Topic 7: [('charge', 0.52417), ('court', 0.46035), ('face', 0.33058), ('plan', 0.1257), ('murder', 0.12235)]
Topic 8: [('charge', 0.53868), ('govt', 0.30554), ('urge', 0.14612), ('fund', 0.08376), ('murder', 0.06437)]
Topic 9: [('win', 0.68415), ('charge', 0.26525), ('council', 0.25092), ('water', 0.12005), ('cup', 0.10833)]
Topic 10: [('council', 0.72539

- 위처럼 LSA를 통해 전체 corpus로부터 주요 topic을 찾을 수 있다. 이를 Topic Modeling라고 한다. 주로 고객의 소리 등과 같이 많은 문서에서 주요 주제를 알아낼 때 사용한다.

## 5) LDA(Latent Dirichlet Allocation)

- 잠재 디리클레 할당
- 토픽 모델링의 또 다른 대표 알고리즘
- 문서들이 토픽들의 혼합으로 구성되어 있으며, 토픽들은 확률 분포에 기반해 단어들을 생성한다고 가정한다. 그리고 데이터가 주어지면, 그에 따라 단어들의 분포로부터 문서가 생성되는 과정을 역추적해 문서의 토픽을 찾아낸다.

### - 시뮬레이션 

- [시뮬레이션 웹사이트](https://lettier.com/projects/lda-topic-modeling/)
- 결과적으로 두 개 행렬이 나오는데, 첫 번째 행렬의 행은 단어 집합의 단어들이고, 열은 topic, 두 번째 행렬의 행은 문서이고, 열을 topic이다.
- LDA는 각 토픽의 단어 분포, 즉 특정 토픽에 특정 단어가 나타낼 확률을 추정하고, 각 문서의 토픽 분포를 추정해낸다.
- 예를 들어 아래 그림의 좌측 Topics에서 초록색 brain이라는 단어가 등장할 확률은 0.04이다.
- 그림 중앙 Documents에는 노랑, 분홍, 하늘색의 세 가지 토픽이 존재하고, 그 중 노랑 노픽이 가장 많다.
- 우측에 Topic proportions and assignments를 보면 문서에 존재하는 토픽의 비율을 시각화한 그래프를 볼 수 있다. 역시 노랑 토픽의 비중이 가장 크다. 즉 이 문서에 노랑 토픽의 단어들이 가장 많이 등장하며, 노랑 토픽일 가능성이 크다.

![image](https://user-images.githubusercontent.com/80008411/135384408-f9f7da7e-2b64-4516-af32-90f8d34c88a2.png)

### - 가정

- LDA는 전체 corpus, 즉 다수의 문서들로부터 토픽을 뽑아내기 위해 하나의 가정을 염두에 둔다. 즉 모든 문서가 작성될 때 그 문서의 작성자는 아래와 같은 생각을 했다는 가정이다.
> 이 문서를 작성하기 위해서 이런 주제들을 넣을 거고, 이런 주제들을 위해서는 이런 단어들을 넣을 것이다.

- 그리고 각 문서를 작성 과정은 다음과 같다고 가정한다.
> (1) 문서에 사용할 단어의 개수 N을 정한다.<br>
(2) 문서에 사용할 토픽의 혼합을 확률 분포에 기반해 결정한다.<br>
(3) 문서에 사용할 각 단어를 다음과 같이 정한다.<br>
(3-1) 토픽 분포에서 토픽 T를 확률적으로 고른다.
(3-2) 선택한 토픽 T에서 단어의 출현 확률 분포에 기반해 문서에 사용할 단어를 고른다.

- 이러한 가정을 바탕으로 LDA는 위 과정의 역으로 추적하는 역공학(reverse engineering)을 수행한다.

### - 직관 이해하기

- 아래와 같은 문서 3개가 있다고 하자.
> 문서1 : 저는 사과랑 바나나를 먹어요<br>
문서2 : 우리는 귀여운 강아지가 좋아요<br>
문서3 : 저의 깜찍하고 귀여운 강아지가 바나나를 먹어요

- 이 문서들로부터 2개의 토픽을 찾는 LDA를 수행한다고 하자. 이때 전처리 과정을 거친 DTM이 LDA의 입력이 되었다고 가정한다.
- LDA는 각 문서의 토픽 분포와 각 토픽 내의 단어 분포를 추정한다. 그 결과는 다음과 같다.

>\<각 문서의 토픽 분포><br>
문서1 : 토픽 A 100%<br>
문서2 : 토픽 B 100%<br>
문서3 : 토픽 B 60%, 토픽 A 40%<br>

>\<각 토픽의 단어 분포><br>
토픽A : 사과 20%, 바나나 40%, 먹어요 40%, 귀여운 0%, 강아지 0%, 깜찍하고 0%, 좋아요 0%<br>
토픽B : 사과 0%, 바나나 0%, 먹어요 0%, 귀여운 33%, 강아지 33%, 깜찍하고 16%, 좋아요 16%

### - LSA와 LDA의 차이
- LSA: DTM을 차원 축소하여 그 축소된 차원에서 근접 단어들을 토픽으로 묶는다.
- LDA: 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합 확률로 추정하여 토픽을 추출한다.

#### * [참고 동영상](https://serviceapi.nmv.naver.com/flash/convertIframeTag.nhn?vid=A008B9E1EAFC02C99F92928155487839090E&outKey=V1210ad4156cf64ce0c6a3e18cecaae499f6528784c999ca6541c3e18cecaae499f65&width=544&height=306)

### - 구현

- LDA는 DTM 또는 TF-IDF를 입력으로 받을 수 있다. 이번에는 TF-IDF를 사용해보자.
- sklearn.decompostition.LatentDirichletAllocation 이용

In [40]:
# 상위 5,000개의 단어만 사용
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


In [41]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.69841093, 0.0335099 , ..., 0.0335099 , 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.0366096 , 0.0366096 , 0.0366096 , ..., 0.67051361, 0.0366096 ,
        0.0366096 ],
       ...,
       [0.02914502, 0.02914502, 0.14077174, ..., 0.02914502, 0.02914502,
        0.26688721],
       [0.02637829, 0.12325014, 0.02638944, ..., 0.21422895, 0.02637829,
        0.0996168 ],
       [0.03376121, 0.03376055, 0.03376055, ..., 0.03376055, 0.50437083,
        0.03376055]])

In [42]:
print(lda_model.components_.shape)

(10, 5000)


In [43]:
terms = tfidf_vectorizer.get_feature_names() # 단어 집합. 5,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
        
get_topics(lda_model.components_, terms)

Topic 1: [('new', 6839.37079), ('government', 6344.47105), ('election', 5419.59529), ('adelaide', 4864.1739), ('home', 4048.41226)]
Topic 2: [('say', 7929.44145), ('change', 4193.89405), ('year', 3924.88995), ('live', 3625.10473), ('market', 3541.15301)]
Topic 3: [('australian', 7667.75985), ('south', 4846.36918), ('perth', 4552.99622), ('2016', 3955.96018), ('open', 3771.68108)]
Topic 4: [('trump', 8187.32772), ('school', 3966.83143), ('jail', 3245.22756), ('women', 3029.28982), ('life', 2998.44275)]
Topic 5: [('police', 5589.78951), ('melbourne', 5299.84238), ('warn', 3577.31091), ('rural', 3521.5736), ('hospital', 3106.7779)]
Topic 6: [('world', 4536.54893), ('sydney', 4406.58731), ('country', 4167.71984), ('years', 3581.99631), ('man', 3520.89397)]
Topic 7: [('charge', 5946.75892), ('day', 5062.31785), ('house', 4481.76928), ('murder', 4065.57534), ('crash', 3793.50261)]
Topic 8: [('australia', 7253.84683), ('attack', 4787.62503), ('north', 3706.11788), ('state', 3658.17043), ('wes

<hr>

## 텍스트 분포를 이용한 비지도 학습 토크나이저 

### 1) 형태소 분석기와 단어 미등록 문제

#### - 형태소 분석기의 필요성

- 한국어는 교착어. 즉 하나의 어절이 하나의 어근(혹은 어간stem)과 각각 단일한 기능을 가지는 하나 이상의 접사(affix)의 결합으로 이루어져 있다. (조사라는 품사는 교착어에만 존재)
- 반면 영어는 단순 띄어쓰기로 토큰화해도 제대로 동작할 수 있다.

In [45]:
# 띄어쓰기로 토큰화
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


In [46]:
# 형태소 분석기로 토큰화
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


#### - 단어 미등록 문제

- 그러나 기존 형태소 분석기는 등록된 단어를 기준으로 형태소를 분류해 내어 새롭게 만들어진 단어를 인식하기 어렵다는 한계가 있다.
- 이를 극복하기 위해, 데이터에서 특정 문자 시퀀스가 함께 자주 등장하고, 앞뒤로 조사 또는 완전히 다른 단어가 등장하는 것을 고려해서 그 문자 시퀀스 자체를 형태소라고 판단할 수 있도록 한 것이 soynlp이다.
- 즉 '모두의연구소'라는 문자열이 자주 연결되어 등장한다면 형태소라고 판단하고, '모두의연구소'라는 단어 앞, 뒤에 '최고', 'AI', '실력'과 같은 독립된 다른 단어들이 계속해서 등장한다면 '모두의연구소'를 형태소로 파악하는 식

In [47]:
# 형태소 분석기에 '모두의연구소'가 하나의 단어로 인식될 수 없음
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


### 2) soynlp

- 품사 태깅, 형태소 분석 등을 지원하는 한국어 형태소 분석기
- 비지도 학습으로 형태소 분석을 하며, 데이터에 자주 등장하는 단어들을 형태소로 분석한다.
- 내부적으로 단어 점수표로 동작하는데, 이 점수는 응집 확률(cohesion probability)과 브랜칭 엔트로피(branching entropy)를 활용한다.
- 비지도학습 형태소 분석기이므로 기존 형태소 분석기와 달리 학습 과정이 필요하다. 전체 corpus로부터 응집 확률과 브랜칭 엔트로피 단어 점수표를 만드는 과정이다.

#### - 응집 확률(Cohesion Probability)

- 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도
- 문자열을 문자 단위로 분리해 내부 문자열을 만드는 과정에서, 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그 다음 문자가 나올 확률을 계산하여 누적 곱을 한 값
- 이 값이 높을수록 전체 코퍼스에서 문자열 시퀀스가 하나의 단어로 등장할 가능성이 높다.

![image](https://user-images.githubusercontent.com/80008411/135401703-8472e176-c391-4cc3-8678-e9d9f6fd998b.png)

- 아래는 '반포한강공원에'라는 길이 7의 문자 시퀀스에 대해 각 내부 문자열의 스코어를 구하는 과정을 나타낸다.

![image](https://user-images.githubusercontent.com/80008411/135401732-b7505b37-8c6e-4908-a817-8ccc249f0244.png)

#### - 구현

- soynlp.DoublespaceLineCorpus()로 문서 분리
- WordExtractor.extract()로 전체 corpus에 대해 단어 점수표 계산

In [48]:
import urllib.request

txt_filename = os.getenv('HOME')+'/aiffel/aiffel_projects/goingdeeper/GD3_topic_modeling/data/2016-10-20.txt'

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename=txt_filename)

('/aiffel/aiffel/aiffel_projects/goingdeeper/GD3_topic_modeling/data/2016-10-20.txt',
 <http.client.HTTPMessage at 0x7ff0d5fde650>)

In [52]:
from soynlp import DoublespaceLineCorpus

# 텍스트 파일을 문서 단위로 분리
corpus = DoublespaceLineCorpus(txt_filename)
print('문서 개수:', len(corpus))

문서 개수: 30091


In [54]:
# 공백이 아닌 문서 3개 출력해보기
i = 0
for doc in corpus:
    if len(doc) > 0:
        print(doc)
        print('-------------------------------\n')
        i += 1
    if i == 3:
        break

19  1990  52 1 22
-------------------------------

오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였

In [55]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 2.577 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [56]:
word_score_table['반포한'].cohesion_forward

0.08838002913645132

In [57]:
word_score_table['반포한강공'].cohesion_forward

0.2972877884078849

In [58]:
word_score_table['반포한강공원'].cohesion_forward

0.37891487632839754

In [59]:
word_score_table["반포한강공원에"].cohesion_forward

0.33492963377557666

결과적으로 '반포한강공원'이 하나의 단어일 확률이 높다.

#### - 브랜칭 엔트로피(Branching Entropy)

- 확률 분포의 엔트로피 값을 사용
- 주어진 문자열에서 다음 문자가 등장할 수 있는 가능성을 판단하는 척도
- 하나의 완성된 단어에 가까워질수록 문맥으로 인해 정확히 예측할 수 있게 되므로 점차 줄어든다.

In [62]:
word_score_table["디"].right_branching_entropy

2.68517802819071

In [60]:
word_score_table["디스"].right_branching_entropy

1.6371694761537934

In [61]:
word_score_table["디스플"].right_branching_entropy

-0.0

In [63]:
word_score_table["디스플레"].right_branching_entropy

-0.0

In [64]:
word_score_table["디스플레이"].right_branching_entropy

3.1400392861792916

- '디스플'까지 오면 엔트로피가 급격이 감소한다. '디스플레이'라는 단어가 등장할 확률이 매우 높기 때문
- 반면 '디스플레이'에서는 다시 값이 급증한다. 그 다음에 조사나 다른 단어가 나올 확률이 높기 때문이다.

#### - LTokenizer

- 띄어쓰기 단위로 잘 나뉜 문장은 L토크나이저를 사용하면 좋다.
- 예를 들어 '공원에'는 '공원 + 에'로, '공부하는'은 '공부 + 하는'으로 나눌 수 있다.
- 이처럼 L토큰 + R토큰으로 나누되, 점수가 가장 높은 L토큰을 찾아내는 분리 기준을 가진다.

In [65]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
l_tokenizer.tokenize('국제사회와 우리의 노력들로 범죄를 척결하자', flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

#### - 최대 점수 토크나이저(MaxScoreTokenizer)

- 띄어쓰기가 되어 있지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아낸다.

In [66]:
from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']